**qAIntum.ai**

# **Quantum Neural Network (QNN) Classifier**

This note is an example of using Photonic Analog (PA) QNN for binary classification. This is an application of the original work ["Continuous Variable Quantum Neural Networks"](https://arxiv.org/abs/1806.06871).

Compared to Classical Neural Networks, PA QNNs have a reduced number of parameters to train and converge faster with fewer epochs. However, this is a quantum algorithm simulated on classical computers hence the training time for quantum circuits tend to be longer than classical models.

The dataset used in this example can be found at https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

This file is organized in the following order:
0. Install and import necessary packages
1. Load and preprocess data (The data from Kaggle is saved as 'financial.csv'.)
2. Data encoding
3. QNN model
   * QNN layer
   * QNN circuit
   * Model building
4. Model training
5. Evaluation

For the open source repository, refer to https://github.com/qaintumai/quantum.

## **0. Intall Packages**

Ensure you have installed the packages detailed in requirements.txt before running the code below.

[Pennylane](https://pennylane.ai/) is a Python based quantum machine learning library by Xanadu. An old version is necessary for this example: v0.29.1.

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pennylane as qml

## **1. Load and Preprocess Data**

Data: 

Data points: 

Number of features: 

Label: 

For the purpose of this experiment, we are using only ___ data points for training and __ for testing.

data source: 

In [3]:
#LOADING DATA
import sys
import os
script_dir = os.getcwd()
data_path = os.path.join(script_dir, '../examples/data/pima-indians-diabetes.csv')
data_path = os.path.normpath(data_path)

# Load the dataset
dataset = np.loadtxt(data_path, delimiter=',')

print("length of dataset: ", len(dataset))

X = dataset[:,0:8]
y = dataset[:,8]

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

print(f"Size of X tensor: {X.size()} and first element of X: {X[0]}")
print(f"Size of Y tensor: {y.size()} and first element of Y: {y[0]}")

length of dataset:  768
Size of X tensor: torch.Size([768, 8]) and first element of X: tensor([  6.0000, 148.0000,  72.0000,  35.0000,   0.0000,  33.6000,   0.6270,
         50.0000])
Size of Y tensor: torch.Size([768, 1]) and first element of Y: tensor([1.])


## **2. Data Encoding**

This step converts classical data into a quantum state by using the data entries as parameters of the quantum gates.

The data encoding gates used are Squeezing, Rotation, Beamsplitter, Displacement Gate, and Kerr Gate. Other gates under "CV operators" in the Pennylane package can be explored.

In [19]:
def encode(x, num_wires):
        """
        Encodes the input data into a quantum state to be operated on using a sequence of quantum gates.

        Parameters:
        x : input data (list or array-like)

        The encoding process uses the following gates in sequence:
        - Squeezing gates: 2*self.num_wires parameters
        - Beamsplitter gates: 2(self.num_wires-1) parameters
        - Rotation gates: self.num_wires parameters
        - Displacement gates: 2*self.num_wires parameters
        - Kerr gates: self.num_wires parameters
          Total: 8*self.num_wires - 2 parameters

        rounds: the number of iterations of the sequence needed to take in all the entries of the input data
                num_features // (8 * self.num_wires - 2)
                We are adding (8 * self.num_wires - 3) as a pad to run one extra round for the remainding data entries.
        """
        num_features = len(x)
        

        # Calculate the number of rounds needed to process all features
        rounds = (num_features + (8 * num_wires - 3)) // (8 * num_wires - 2)

        for j in range(rounds):
            start_idx = j * (8 * num_wires - 2)

            # Squeezing gates
            for i in range(num_wires):
                # for each wire, the number of parameters are i*2
                idx = start_idx + i * 2
                if idx + 1 < num_features:
                    qml.Squeezing(x[idx], x[idx + 1], wires=i)

            # Beamsplitter gates
            for i in range(num_wires - 1):
                # start_index + Squeezing gates, and then i*2 parameters for each gate
                idx = start_idx + num_wires * 2 + i * 2
                if idx + 1 < num_features:
                    qml.Beamsplitter(x[idx], x[idx + 1], wires=[i % num_wires, (i + 1) % num_wires])

            # Rotation gates
            for i in range(num_wires):
                # start_index + Squeezing gates + Beamsplitters, and then i parameters for each gate
                idx = start_idx + num_wires * 2 + (num_wires - 1) * 2 + i
                if idx < num_features:
                    qml.Rotation(x[idx], wires=i)

            # Displacement gates
            for i in range(num_wires):
                # start_index + Squeezing gates + Beamsplitters + Rotation gates, and then i*2 parameters for each gate
                idx = start_idx + num_wires * 2 + (num_wires - 1) * 2 + num_wires + i * 2
                if idx + 1 < num_features:
                    qml.Displacement(x[idx], x[idx + 1], wires=i)

            # Kerr gates
            for i in range(num_wires):
                # start_index + Squeezing gates + Beamsplitters + Rotation gates + Displacement gates, and then i parameters for each gate
                idx = start_idx + num_wires * 2 + (num_wires - 1) * 2 + num_wires + num_wires * 2 + i
                if idx < num_features:
                    qml.Kerr(x[idx], wires=i)

## **3. QNN Model**
To build a model, we need to
* define a layer
* build a circuit with the defined layer
* build a model with the defined circuit.

### **3.1 QNN Layer**
This in a faithful implementation of classical neural networks:
* Weight matrix: Interferometer 1 + Squeezing + Interferometer 2
* Bias addition: Displacement gate
* Nonlinear activation function: Kerr gate

In [20]:
def apply(v, num_wires):
        """
        Applies the quantum neural network layer with the given parameters.

        Parameters:
        - v (list or array): List or array of parameters for the quantum gates.

        Returns:
        - None
        """
        num_params = len(v)

        # Interferometer 1
        for i in range(num_wires - 1):
            idx = i * 2
            if idx + 1 < num_params:
                theta = v[idx]
                phi = v[idx + 1]
                qml.Beamsplitter(theta, phi, wires=[i % num_wires, (i + 1) % num_wires])

        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + i
            if idx < num_params:
                qml.Rotation(v[idx], wires=i)

        # Squeezers
        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + i
            if idx < num_params:
                qml.Squeezing(v[idx], 0.0, wires=i)

        # Interferometer 2
        for i in range(num_wires - 1):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + i * 2
            if idx + 1 < num_params:
                theta = v[idx]
                phi = v[idx + 1]
                qml.Beamsplitter(theta, phi, wires=[i % num_wires, (i + 1) % num_wires])

        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + i
            if idx < num_params:
                qml.Rotation(v[idx], wires=i)

        # Bias addition
        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + num_wires + i
            if idx < num_params:
                qml.Displacement(v[idx], 0.0, wires=i)

        # Non-linear activation function
        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + num_wires + num_wires + i
            if idx < num_params:
                qml.Kerr(v[idx], wires=i)


**Weight Initializer**

Randomly initialized values are used as initial parameters of the QNN circuit.

In [14]:
def init_weights(layers, num_wires, active_sd=0.0001, passive_sd=0.1):
    """
    This is a weight vector initializer.
    Input: number of layers, number of wires
    Output: concatenated weight vector
    """
    M = (num_wires - 1) * 2 + num_wires  # Number of interferometer parameters

    int1_weights = np.random.normal(size=[layers, M], scale=passive_sd) #beamsplitters and rotations
    s_weights = np.random.normal(size=[layers, num_wires], scale=active_sd) #squeezers
    int2_weights = np.random.normal(size=[layers, M], scale=passive_sd) #beamsplitters and rotations
    dr_weights = np.random.normal(size=[layers, num_wires], scale=active_sd) #displacement
    k_weights = np.random.normal(size=[layers, num_wires], scale=active_sd) #Kerr

    weights = np.concatenate(
        [int1_weights, s_weights, int2_weights, dr_weights, k_weights], axis=1)

    return weights

### **3.2 QNN Circuit**

For building a PA circuit as opposed to a qubit-based circuit, we have to choose "strawberryfields.fock" as device.

In [21]:
num_wires = 8
num_basis = 2

# select a device
dev = qml.device("strawberryfields.fock", wires=num_wires, cutoff_dim=num_basis)

@qml.qnode(dev, interface="torch")
def quantum_nn(inputs, var):
    """
    This is a quantum circuit composed of a data encoding circuit and a QNN circuit.
    Input: classical data (inputs), quantum parameters (var)
    Output: quantum state, converted to classical data after measurement
    """
    # convert classical inputs into quantum states
    encode(inputs, num_wires)
    print("Encoded data")

    # iterative quantum layers
    for v in var:
        apply(v, num_wires)

    # measure the resulting state and return
    return qml.expval(qml.X(0))

### **3.3 Model building**

In [22]:
num_layers = 2

def get_model(num_wires, num_layers):
    """
    This is a model building function.
    Input: number of modes, number of layers
    Output: PyTorch model
    """
    weights = init_weights(num_layers, num_wires)
    shape_tup = weights.shape
    weight_shapes = {'var': shape_tup}
    qlayer = qml.qnn.TorchLayer(quantum_nn, weight_shapes)
    model = torch.nn.Sequential(qlayer)
    return model

In [23]:
model = get_model(num_wires, num_layers)
print(model.parameters())
print(model)

<generator object Module.parameters at 0x12b788e40>
Sequential(
  (0): <Quantum Torch Layer: func=quantum_nn>
)


## **4. Model Training**

In [24]:

#TRAINING MODEL
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.01)

n_epochs = 1
batch_size = 2
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        print("====================================================================================")
        print(f"+++++++++++++++Batch number: {i} in Epoch {epoch}++++++++++++++++++++++++++++++++++")
        print("====================================================================================")

        Xbatch = X[i:i+batch_size]
        print(f"Size of Xbatch: {Xbatch.size()}")
        y_pred = model(Xbatch).reshape(-1, 1)  
        print(f"Size of y_pred: {y_pred.size()}, Values: {y_pred}")

        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        print(f"loss = {loss}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Finished epoch {epoch}, latest loss {loss}')


+++++++++++++++Batch number: 0 in Epoch 0++++++++++++++++++++++++++++++++++
Size of Xbatch: torch.Size([2, 8])
Encoded data
Encoded data
Size of y_pred: torch.Size([2, 1]), Values: tensor([[0.],
        [0.]], grad_fn=<ViewBackward0>)
loss = 50.0
+++++++++++++++Batch number: 2 in Epoch 0++++++++++++++++++++++++++++++++++
Size of Xbatch: torch.Size([2, 8])
Encoded data
Encoded data
Size of y_pred: torch.Size([2, 1]), Values: tensor([[0.],
        [0.]], grad_fn=<ViewBackward0>)
loss = 50.0
+++++++++++++++Batch number: 4 in Epoch 0++++++++++++++++++++++++++++++++++
Size of Xbatch: torch.Size([2, 8])
Encoded data
Encoded data
Size of y_pred: torch.Size([2, 1]), Values: tensor([[0.],
        [0.]], grad_fn=<ViewBackward0>)
loss = 50.0
+++++++++++++++Batch number: 6 in Epoch 0++++++++++++++++++++++++++++++++++
Size of Xbatch: torch.Size([2, 8])
Encoded data
Encoded data
Size of y_pred: torch.Size([2, 1]), Values: tensor([[0.],
        [0.]], grad_fn=<ViewBackward0>)
loss = 50.0
++++++++++++

KeyboardInterrupt: 

## **5. Evaluation**

In [ ]:
# EVALUATING
with torch.no_grad():
    y_pred = model(X)

    print(f"Size of y_pred (evaluation): {y_pred.size()}, Values: {y_pred}")

accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy: {accuracy}")